In [20]:
# -------------------------------------------------------------------------------------------------
#  Copyright (C) 2015-2024 Nautech Systems Pty Ltd. All rights reserved.
#  https://nautechsystems.io
#
#  Licensed under the GNU Lesser General Public License Version 3.0 (the "License");
#  You may not use this file except in compliance with the License.
#  You may obtain a copy of the License at https://www.gnu.org/licenses/lgpl-3.0.en.html
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
# -------------------------------------------------------------------------------------------------

from os import PathLike
from pathlib import Path
import requests

from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.persistence.wranglers import QuoteTickDataWrangler
from nautilus_trader.test_kit.providers import CSVTickDataLoader
from nautilus_trader.test_kit.providers import TestInstrumentProvider


ROOT = Path("D:\Projects/My Own Project/NautilusTrader")
CATALOG_DIR = ROOT / "catalog"
CATALOG_DIR.mkdir(exist_ok=True)


def load_fx_hist_data(
    filename: str,
    currency: str,
    catalog_path: PathLike[str] | str,
) -> None:
    instrument = TestInstrumentProvider.default_fx_ccy(currency)
    wrangler = QuoteTickDataWrangler(instrument)

    df = CSVTickDataLoader.load(filename, index_col=0, datetime_format="%Y%m%d %H%M%S%f")
    df.columns = ["bid_price", "ask_price", "size"]
    print(df)

    print("Preparing ticks...")
    ticks = wrangler.process(df)

    print("Writing data to catalog...")
    catalog = ParquetDataCatalog(catalog_path)
    catalog.write_data([instrument])
    catalog.write_data(ticks)

    print("Done")


def download(url: str) -> None:
    filename = url.rsplit("/", maxsplit=1)[1]
    with open(filename, "wb") as f:
        f.write(requests.get(url).content)


def main():
    # Download raw data
    download(
        "https://raw.githubusercontent.com/nautechsystems/nautilus_data/main/raw_data/fx_hist_data/DAT_ASCII_EURUSD_T_202001.csv.gz",
    )
    load_fx_hist_data(
        filename="DAT_ASCII_EURUSD_T_202001.csv.gz",
        currency="EUR/USD",
        catalog_path=CATALOG_DIR,
    )


if __name__ == "__main__":
    main()

C:\Users\baloochyb\AppData\Roaming\Python\Python311\site-packages\nautilus_trader\persistence\loaders.py:56: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(


                         bid_price  ask_price  size
20200101 170000065                                 
2020-01-01 17:00:10.447    1.12120    1.12192     0
2020-01-01 17:00:10.498    1.12117    1.12161     0
2020-01-01 17:00:12.579    1.12120    1.12161     0
2020-01-01 17:00:12.630    1.12120    1.12172     0
2020-01-01 17:00:12.839    1.12120    1.12171     0
...                            ...        ...   ...
2020-01-31 16:59:53.963    1.10945    1.10961     0
2020-01-31 16:59:54.064    1.10945    1.10957     0
2020-01-31 16:59:54.711    1.10945    1.10960     0
2020-01-31 16:59:55.922    1.10945    1.10952     0
2020-01-31 16:59:56.198    1.10944    1.10952     0

[1637768 rows x 3 columns]
Preparing ticks...
Writing data to catalog...
Done


In [21]:
from nautilus_trader.persistence.catalog import ParquetDataCatalog


# You can also use `ParquetDataCatalog.from_env()` which will use the `NAUTILUS_PATH` environment variable 
# catalog = ParquetDataCatalog.from_env()
catalog = ParquetDataCatalog("./catalog")
catalog.instruments()

[CurrencyPair(id=EUR/USD.SIM, raw_symbol=EUR/USD, asset_class=FX, instrument_class=SPOT, quote_currency=USD, is_inverse=False, price_precision=5, price_increment=0.00001, size_precision=0, size_increment=1, multiplier=1, lot_size=1000, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None)]

In [23]:
from nautilus_trader.core.message import Event
from nautilus_trader.indicators.macd import MovingAverageConvergenceDivergence
from nautilus_trader.model.data import QuoteTick
from nautilus_trader.model.enums import PriceType
from nautilus_trader.model.enums import PositionSide
from nautilus_trader.model.enums import OrderSide
from nautilus_trader.model.events import PositionOpened
from nautilus_trader.model.identifiers import InstrumentId
from nautilus_trader.model.objects import Quantity
from nautilus_trader.model.position import Position
from nautilus_trader.trading.strategy import Strategy, StrategyConfig


class MACDConfig(StrategyConfig):
    instrument_id: InstrumentId
    fast_period: int = 12
    slow_period: int = 26
    trade_size: int = 1_000_000
    entry_threshold: float = 0.00010


class MACDStrategy(Strategy):
    def __init__(self, config: MACDConfig) -> None:
        super().__init__(config=config)
        # Our "trading signal"
        self.macd = MovingAverageConvergenceDivergence(
            fast_period=config.fast_period,
            slow_period=config.slow_period,
            price_type=PriceType.MID,
        )
        # We copy some config values onto the class to make them easier to reference later on
        self.entry_threshold = config.entry_threshold
        self.instrument_id = config.instrument_id
        self.trade_size = Quantity.from_int(config.trade_size)

        # Convenience
        self.position: Position | None = None

    def on_start(self):
        self.subscribe_quote_ticks(instrument_id=self.instrument_id)

    def on_stop(self):
        self.close_all_positions(self.instrument_id)
        self.unsubscribe_quote_ticks(instrument_id=self.instrument_id)

    def on_quote_tick(self, tick: QuoteTick):
        # You can register indicators to receive quote tick updates automatically,
        # here we manually update the indicator to demonstrate the flexibility available.
        self.macd.handle_quote_tick(tick)

        if not self.macd.initialized:
            return  # Wait for indicator to warm up
        
        # self._log.info(f"{self.macd.value=}:%5d")
        self.check_for_entry()
        self.check_for_exit()

    def on_event(self, event):
        if isinstance(event, PositionOpened):
            self.position = self.cache.position(event.position_id)

    def check_for_entry(self):
        # If MACD line is above our entry threshold, we should be LONG
        if self.macd.value > self.entry_threshold:
            if self.position and self.position.side == PositionSide.LONG:
                return  # Already LONG

            order = self.order_factory.market(
                instrument_id=self.instrument_id,
                order_side=OrderSide.BUY,
                quantity=self.trade_size,
            )
            self.submit_order(order)
        # If MACD line is below our entry threshold, we should be SHORT
        elif self.macd.value < -self.entry_threshold:
            if self.position and self.position.side == PositionSide.SHORT:
                return  # Already SHORT

            order = self.order_factory.market(
                instrument_id=self.instrument_id,
                order_side=OrderSide.SELL,
                quantity=self.trade_size,
            )
            self.submit_order(order)

    def check_for_exit(self):
        # If MACD line is above zero then exit if we are SHORT
        if self.macd.value >= 0.0:
            if self.position and self.position.side == PositionSide.SHORT:
                self.close_position(self.position)
        # If MACD line is below zero then exit if we are LONG
        else:
            if self.position and self.position.side == PositionSide.LONG:
                self.close_position(self.position)

    def on_dispose(self):
        pass  # Do nothing else


In [24]:
from nautilus_trader.config import BacktestVenueConfig


venue = BacktestVenueConfig(
    name="SIM",
    oms_type="NETTING",
    account_type="MARGIN",
    base_currency="USD",
    starting_balances=["1_000_000 USD"],
)

In [25]:
instruments = catalog.instruments()
instruments

[CurrencyPair(id=EUR/USD.SIM, raw_symbol=EUR/USD, asset_class=FX, instrument_class=SPOT, quote_currency=USD, is_inverse=False, price_precision=5, price_increment=0.00001, size_precision=0, size_increment=1, multiplier=1, lot_size=1000, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None)]

In [26]:
from nautilus_trader.config import BacktestDataConfig
from nautilus_trader.model.data import QuoteTick


data = BacktestDataConfig(
    catalog_path=str(catalog.path),
    data_cls=QuoteTick,
    instrument_id=instruments[0].id,
    end_time="2020-01-10",
)

In [27]:
from nautilus_trader.config import BacktestEngineConfig
from nautilus_trader.config import ImportableStrategyConfig
from nautilus_trader.config import LoggingConfig


engine = BacktestEngineConfig(
    strategies=[
        ImportableStrategyConfig(
            strategy_path="__main__:MACDStrategy",
            config_path="__main__:MACDConfig",
            config=dict(
              instrument_id=instruments[0].id,
              fast_period=12,
              slow_period=26,
            ),
        )
    ],
    logging=LoggingConfig(log_level="ERROR"),  # Lower to `INFO` to see more logging about orders, events, etc.
)

In [28]:
from nautilus_trader.config import BacktestRunConfig


config = BacktestRunConfig(
    engine=engine,
    venues=[venue],
    data=[data],
)

In [29]:
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.results import BacktestResult


node = BacktestNode(configs=[config])

 # Runs one or many configs synchronously
results: list[BacktestResult] = node.run()

In [30]:
from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.model.identifiers import Venue


engine: BacktestEngine = node.get_engine(config.id)

engine.trader.generate_order_fills_report()

,trader_id,strategy_id,instrument_id,venue_order_id,position_id,account_id,last_trade_id,type,side,quantity,...,order_list_id,linked_order_ids,parent_order_id,exec_algorithm_id,exec_algorithm_params,exec_spawn_id,tags,init_id,ts_init,ts_last
client_order_id,,,,,,,,,,,,,,,,,,,,,
O-20200103-1000-001-000-1,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-001,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-001,MARKET,BUY,1000000,...,None,None,None,None,None,None,None,f90cd79e-c816-403d-86d2-27f9bda54bd7,2020-01-03 10:00:00.242000+00:00,2020-01-03 10:00:00.242000+00:00
O-20200103-1000-001-000-2,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-002,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-002,MARKET,SELL,1000000,...,None,None,None,None,None,None,None,9e99466c-5cf1-4ed1-8634-4bfdd1176a7c,2020-01-03 10:00:04.136000+00:00,2020-01-03 10:00:04.136000+00:00
O-20200103-1000-001-000-3,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-003,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-003,MARKET,SELL,1000000,...,None,None,None,None,None,None,None,346afb6c-f0c7-486b-812e-33353596dadd,2020-01-03 10:00:06.093000+00:00,2020-01-03 10:00:06.093000+00:00
O-20200103-1000-001-000-4,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-004,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-004,MARKET,BUY,1000000,...,None,None,None,None,None,None,None,ff8e6a5f-255a-494e-a82f-1be920e61387,2020-01-03 10:00:07.548000+00:00,2020-01-03 10:00:07.548000+00:00
O-20200103-1652-001-000-5,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-005,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-005,MARKET,SELL,1000000,...,None,None,None,None,None,None,None,97078eca-0927-4d72-9093-e8d2bbc6490f,2020-01-03 16:52:30.901000+00:00,2020-01-03 16:52:30.901000+00:00
O-20200103-1652-001-000-6,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-006,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-006,MARKET,BUY,1000000,...,None,None,None,None,None,None,None,cd01f3ac-f250-4f34-bd1a-c7141f56a05d,2020-01-03 16:52:46.001000+00:00,2020-01-03 16:52:46.001000+00:00
O-20200105-1700-001-000-7,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-007,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-007,MARKET,BUY,1000000,...,None,None,None,None,None,None,None,beb34a38-2862-4052-9340-56aea5f4315f,2020-01-05 17:00:32.758000+00:00,2020-01-05 17:00:32.758000+00:00
O-20200105-1705-001-000-8,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-008,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-008,MARKET,SELL,1000000,...,None,None,None,None,None,None,None,90f926a9-016e-4dd9-97f4-0a00ea36c5b3,2020-01-05 17:05:30.628000+00:00,2020-01-05 17:05:30.628000+00:00
O-20200108-0313-001-000-10,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-010,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-010,MARKET,BUY,1000000,...,None,None,None,None,None,None,None,0d44925c-7508-4f7c-bb06-190cfd7be64d,2020-01-08 03:13:16.947000+00:00,2020-01-08 03:13:16.947000+00:00


In [31]:
engine.trader.generate_positions_report()

,trader_id,strategy_id,instrument_id,account_id,opening_order_id,closing_order_id,entry,side,peak_qty,ts_opened,ts_last,ts_closed,duration_ns,avg_px_open,avg_px_close,commissions,realized_return,realized_pnl
position_id,,,,,,,,,,,,,,,,,,
EUR/USD.SIM-MACDStrategy-000-2ec9971a-5588-4b0d-8c56-4a036517ef75,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200103-1000-001-000-1,O-20200103-1000-001-000-2,BUY,FLAT,1000000,2020-01-03 10:00:00.242000+00:00,1578045604136000000,2020-01-03 10:00:04.136000+00:00,3894000000,1.11644,1.11635,['44.66 USD'],-8e-05,-134.66 USD
EUR/USD.SIM-MACDStrategy-000-066bad45-9b4e-4378-987b-1b72ab140e42,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200103-1000-001-000-3,O-20200103-1000-001-000-4,SELL,FLAT,1000000,2020-01-03 10:00:06.093000+00:00,1578045607548000000,2020-01-03 10:00:07.548000+00:00,1455000000,1.11604,1.11637,['44.65 USD'],-0.0003,-374.65 USD
EUR/USD.SIM-MACDStrategy-000-173c5c30-7466-4919-9d0e-b3bfd9fe9a50,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200103-1652-001-000-5,O-20200103-1652-001-000-6,SELL,FLAT,1000000,2020-01-03 16:52:30.901000+00:00,1578070366001000000,2020-01-03 16:52:46.001000+00:00,15100000000,1.11534,1.11543,['44.62 USD'],-8e-05,-134.62 USD
EUR/USD.SIM-MACDStrategy-000-15363cc5-1e2c-46a3-ab58-eb68721a86b5,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200105-1700-001-000-7,O-20200105-1705-001-000-8,BUY,FLAT,1000000,2020-01-05 17:00:32.758000+00:00,1578243930628000000,2020-01-05 17:05:30.628000+00:00,297870000000,1.11703,1.11652,['44.67 USD'],-0.00046,-554.67 USD
EUR/USD.SIM-MACDStrategy-000-fe2ddc6b-e72e-4bdd-b9dc-5f3a3ebfabc4,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200108-0313-001-000-9,O-20200108-0313-001-000-10,SELL,FLAT,1000000,2020-01-08 03:13:12.834000+00:00,1578453196947000000,2020-01-08 03:13:16.947000+00:00,4113000000,1.11403,1.1136,['44.55 USD'],0.00039,385.45 USD
EUR/USD.SIM-MACDStrategy-000-8ba643eb-37a9-49de-ac44-46a93705469c,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200108-0621-001-000-11,O-20200108-0621-001-000-12,SELL,FLAT,1000000,2020-01-08 06:21:39.588000+00:00,1578464517525000000,2020-01-08 06:21:57.525000+00:00,17937000000,1.11231,1.11211,['44.49 USD'],0.00018,155.51 USD
EUR/USD.SIM-MACDStrategy-000-5c9e711d-1f4e-4eff-9635-0749770fd8fe,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200108-0815-001-000-13,O-20200108-0815-001-000-14,SELL,FLAT,1000000,2020-01-08 08:15:01.502000+00:00,1578471314869000000,2020-01-08 08:15:14.869000+00:00,13367000000,1.11121,1.1112,['44.44 USD'],1e-05,-34.44 USD
EUR/USD.SIM-MACDStrategy-000,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200108-1726-001-000-15,O-20200108-1726-001-000-16,BUY,FLAT,1000000,2020-01-08 17:26:44.662000+00:00,1578504410316000000,2020-01-08 17:26:50.316000+00:00,5654000000,1.11164,1.11121,['44.45 USD'],-0.00039,-474.45 USD


In [32]:
engine.trader.generate_account_report(Venue("SIM"))

,total,locked,free,currency,account_id,account_type,base_currency,margins,reported,info
2020-01-01 17:00:10.447000+00:00,1000000.00,0.00,1000000.00,USD,SIM-001,MARGIN,USD,[],True,{}
2020-01-03 10:00:00.242000+00:00,999977.67,0.00,999977.67,USD,SIM-001,MARGIN,USD,[],False,{}
2020-01-03 10:00:00.242000+00:00,999977.67,33515.53,966462.14,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '0.00', ...",False,{}
2020-01-03 10:00:04.136000+00:00,999865.34,33515.53,966349.81,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '0.00', ...",False,{}
2020-01-03 10:00:04.136000+00:00,999865.34,0.00,999865.34,USD,SIM-001,MARGIN,USD,[],False,{}
2020-01-03 10:00:06.093000+00:00,999843.02,0.00,999843.02,USD,SIM-001,MARGIN,USD,[],False,{}
2020-01-03 10:00:06.093000+00:00,999843.02,33503.52,966339.50,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '0.00', ...",False,{}
2020-01-03 10:00:07.548000+00:00,999490.69,33503.52,965987.17,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '0.00', ...",False,{}
2020-01-03 10:00:07.548000+00:00,999490.69,0.00,999490.69,USD,SIM-001,MARGIN,USD,[],False,{}
2020-01-03 16:52:30.901000+00:00,999468.38,0.00,999468.38,USD,SIM-001,MARGIN,USD,[],False,{}
